# Práctica 9: interacciones entre partículas 

En la novena práctica trabajamos con un modelo simplificado para los fenómenos de atracción y repulsión de física (o química, de hecho). Supongamos que contemos con $n$ partículas que habitan un cuadro unitario bidimensional y que cada partícula tiene una carga eléctrica, distribuida independientemente e normalmente al azar entre $[-1,1]$. Cargas de un mismo signo producirán una repulsión mientras cargas opuestas resultan en una atracción — la magnitud de la fuerza estará proporcional a la diferencia de magnitud de las cargas (mayores diferencias resultando en fuerzas mayores), y además la fuerza será inversamente proporcional a la distancia euclideana entre las partículas (éstas son reglas inventadas de interacción para efectos de demostración). Vamos a comenzar creando y posicionando las partículas, usando la distribución normal (posteriormente normalizada al cuadro unitario) para las coordenadas $x$ y $y$.

$n$ es la cantidad de partículas generadas. 

En **p** se guardan las coordenadas de las $n$ partículas.

La columna **c** en el data.frame **p** son las *cargas* que posee cada una de las partículas.

In [49]:
library(rasterVis)
library(latticeExtra)
library(lattice)
library(sp)
library(viridisLite)
library(reshape2) 

Código para crear animaciones.

In [394]:
n <- 100
masas=c(0.2,0.4,0.6,0.8,1.0)
a=c(0,0,0,0,0)
b=c(0,0,0,0,0)
p <- data.frame(x = rnorm(n), y=rnorm(n), c=rnorm(n), m=sample(masas,size = n, replace = TRUE))
xmax <- max(p$x)
xmin <- min(p$x)
p$x <- (p$x - xmin) / (xmax - xmin) # ahora son de 0 a 1
ymax <- max(p$y)
ymin <- min(p$y)
p$y <- (p$y - ymin) / (ymax - ymin) # las y tambien
cmax <- max(p$c)
cmin <- min(p$c)
p$c <- 2 * (p$c - cmin) / (cmax - cmin) - 1 # cargas son entre -1 y 1
p$g <- round(5 * p$c) # coloreamos segun la carga a 11 niveles de -5 a 5
paso <- floor(256 / 10)
niveles <- seq(0, 255, paso)
colores <- rgb(niveles, rep(0, 11), rev(niveles), max=255)
eps <- 0.001
fuerza <- function(i) {
    xi <- p[i,]$x
    yi <- p[i,]$y
    ci <- p[i,]$c
    fx <- 0
    fy <- 0
    for (j in 1:n) {
        cj <- p[j,]$c
        dir <- (-1)^(1 + 1 * (ci * cj < 0))
        dx <- xi - p[j,]$x
        dy <- yi - p[j,]$y
        factor <- dir * abs(ci - cj) / (sqrt(dx^2 + dy^2) + eps)
        fx <- fx - dx * factor
        fy <- fy - dy * factor
    }
    return(c(fx, fy))
}
suppressMessages(library(doParallel))
registerDoParallel(makeCluster(detectCores() - 1))
system("rm -f p9_t*.png") # borramos anteriores (requiere bash)
tmax <- 100
digitos <- floor(log(tmax, 10)) + 1
tl <- "0"
while (nchar(tl) < digitos) {
    tl <- paste("0", tl, sep="")
}

    puntos2 <- SpatialPoints(matrix(c(p[p$m==0.2,]$x, p[p$m==0.2,]$y), length(p[p$m==0.2,]$x), byrow=FALSE))
    puntos4 <- SpatialPoints(matrix(c(p[p$m==0.4,]$x, p[p$m==0.4,]$y), length(p[p$m==0.4,]$x), byrow=FALSE))
    puntos6 <- SpatialPoints(matrix(c(p[p$m==0.6,]$x, p[p$m==0.6,]$y), length(p[p$m==0.6,]$x), byrow=FALSE))
    puntos8 <- SpatialPoints(matrix(c(p[p$m==0.8,]$x, p[p$m==0.8,]$y), length(p[p$m==0.8,]$x), byrow=FALSE))
    puntos10 <- SpatialPoints(matrix(c(p[p$m==1.0,]$x, p[p$m==1.0,]$y), length(p[p$m==1.0,]$x), byrow=FALSE))
png(paste("p9_t", tl, ".png", sep=""))
    xyplot(a~b, type="n", xlab="X", ylab="Y", xlim=c(-0.1, 1.1), ylim=c(-0.1, 1.1),
       key = list(space="right",rows=2, 
       text=list(lab=c("Carga","-5","-4","-3","-2","-1","0","1","2","3","4","5","","Masa","0.2","0.4","0.6","0.8","1.0")),
       points=list(pch=c(16,16,16,16,16), col=c("white",colores, "white","white","black","black","black","black","black"), cex=c(2,2,2,2,2,2,2,2,2,2,2,2,2,2,1.8,2.2,2.6,3,3.4))))+
    layer(sp.points(puntos10, pch=16, cex=3.4, col=colores[p$g+6]))+
    layer(sp.points(puntos8, pch=16, cex=3, col=colores[p$g+6]))+
    layer(sp.points(puntos6, pch=16, cex=2.6, col=colores[p$g+6]))+
    layer(sp.points(puntos4, pch=16, cex=2.2, col=colores[p$g+6]))+
    layer(sp.points(puntos2, pch=16, cex=1.8, col=colores[p$g+6]))
graphics.off()

for (iter in 1:tmax) {
    f <- foreach(i = 1:n, .combine=c) %dopar% fuerza(i)
    delta <- 0.02 / max(abs(f)) # que nadie desplace una paso muy largo
    p$x <- foreach(i = 1:n, .combine=c) %dopar% max(min(p[i,]$x + delta * f[c(TRUE, FALSE)][i]/p[i,]$m, 1), 0)
    p$y <- foreach(i = 1:n, .combine=c) %dopar% max(min(p[i,]$y + delta * f[c(FALSE, TRUE)][i]/p[i,]$m, 1), 0)
    tl <- paste(iter, "", sep="")
    while (nchar(tl) < digitos) {
        tl <- paste("0", tl, sep="")
    }
    
    
    puntos2 <- SpatialPoints(matrix(c(p[p$m==0.2,]$x, p[p$m==0.2,]$y), length(p[p$m==0.2,]$x), byrow=FALSE))
    puntos4 <- SpatialPoints(matrix(c(p[p$m==0.4,]$x, p[p$m==0.4,]$y), length(p[p$m==0.4,]$x), byrow=FALSE))
    puntos6 <- SpatialPoints(matrix(c(p[p$m==0.6,]$x, p[p$m==0.6,]$y), length(p[p$m==0.6,]$x), byrow=FALSE))
    puntos8 <- SpatialPoints(matrix(c(p[p$m==0.8,]$x, p[p$m==0.8,]$y), length(p[p$m==0.8,]$x), byrow=FALSE))
    puntos10 <- SpatialPoints(matrix(c(p[p$m==1.0,]$x, p[p$m==1.0,]$y), length(p[p$m==1.0,]$x), byrow=FALSE))
   

    imagen <- xyplot(a~b, type="n", xlab="X", ylab="Y", xlim=c(-0.1, 1.1), ylim=c(-0.1, 1.1),
       key = list(space="right",rows=2, 
       text=list(lab=c("Carga","-5","-4","-3","-2","-1","0","1","2","3","4","5","","Masa","0.2","0.4","0.6","0.8","1.0")),
       points=list(pch=c(16,16,16,16,16), col=c("white",colores, "white","white","black","black","black","black","black"), 
                   cex=c(2,2,2,2,2,2,2,2,2,2,2,2,2,2,1.8,2.2,2.6,3,3.4))))+
                layer(sp.points(puntos10, pch=16, cex=3.4, col=colores[p$g+6]))+
                layer(sp.points(puntos8, pch=16, cex=3, col=colores[p$g+6]))+
                layer(sp.points(puntos6, pch=16, cex=2.6, col=colores[p$g+6]))+
                layer(sp.points(puntos4, pch=16, cex=2.2, col=colores[p$g+6]))+
                layer(sp.points(puntos2, pch=16, cex=1.8, col=colores[p$g+6]))
    
    png(paste("p9_t", tl, ".png", sep=""))    
    print(imagen)
    graphics.off()
}
stopImplicitCluster()
system("convert -delay 25 -size 300x300 p9_t*.png -loop 0 p9.gif") # creamos animacion con ImageMagick

Código modificado

In [418]:
n <- 500
masas=c(0.2,0.4,0.6,0.8,1.0)
a=c(0,0,0,0,0)
b=c(0,0,0,0,0)
p <- data.frame(x = rnorm(n), y=rnorm(n), c=rnorm(n), m=sample(masas,size = n, replace = TRUE))
d <- p
xmax <- max(p$x)
xmin <- min(p$x)
p$x <- (p$x - xmin) / (xmax - xmin) # ahora son de 0 a 1
ymax <- max(p$y)
ymin <- min(p$y)
p$y <- (p$y - ymin) / (ymax - ymin) # las y tambien
cmax <- max(p$c)
cmin <- min(p$c)
p$c <- 2 * (p$c - cmin) / (cmax - cmin) - 1 # cargas son entre -1 y 1
p$g <- round(5 * p$c) # coloreamos segun la carga a 11 niveles de -5 a 5
paso <- floor(256 / 10)
niveles <- seq(0, 255, paso)
colores <- rgb(niveles, rep(0, 11), rev(niveles), max=255)
eps <- 0.001
fuerza <- function(i) {
    xi <- p[i,]$x
    yi <- p[i,]$y
    ci <- p[i,]$c
    fx <- 0
    fy <- 0
    for (j in 1:n) {
        cj <- p[j,]$c
        dir <- (-1)^(1 + 1 * (ci * cj < 0))
        dx <- xi - p[j,]$x
        dy <- yi - p[j,]$y
        factor <- dir * abs(ci - cj) / (sqrt(dx^2 + dy^2) + eps)
        fx <- fx - dx * factor
        fy <- fy - dy * factor
    }
    return(c(fx, fy))
}

suppressMessages(library(doParallel))
registerDoParallel(makeCluster(detectCores() - 1))
data <- data.frame()
tmax <- 100
for (iter in 1:tmax) {
    a <- p$x
    b <- p$y
    f <- foreach(i = 1:n, .combine=c) %dopar% fuerza(i)
    delta <- 0.02 / max(abs(f)) # que nadie desplace una paso muy largo
    p$x <- foreach(i = 1:n, .combine=c) %dopar% max(min(p[i,]$x + delta * f[c(TRUE, FALSE)][i]/p[i,]$m, 1), 0)
    p$y <- foreach(i = 1:n, .combine=c) %dopar% max(min(p[i,]$y + delta * f[c(FALSE, TRUE)][i]/p[i,]$m, 1), 0)
    dist <- sqrt((a-p$x)^2+(b-p$y)^2)
    data <- rbind(data, dist)
}
names(data) <- c(1:n)

stopImplicitCluster()


In [419]:
v <- c()
for (i in 1:n){
    v[i] <- sum(data[,i])/(n-1)
}

In [420]:
d$v <- v

In [421]:
xtable::xtable(print(head(d)))

           x          y          c   m           v
1 -1.0433343  2.8686023 -0.7423646 1.0 0.001890412
2  0.7119557  2.0243932  1.4867207 0.6 0.004618382
3 -0.3596206 -1.9417118 -0.8000972 0.2 0.009067467
4 -2.5739024 -0.8338202 -0.8613619 1.0 0.002091908
5 -0.5364081 -0.8442296  0.5731916 0.2 0.008063160
6  0.9836179 -0.9878642  0.3748360 1.0 0.001528506


,x,y,c,m,v
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,-1.0433343,2.8686023,-0.7423646,1.0,0.001890412
2,0.7119557,2.0243932,1.4867207,0.6,0.004618382
3,-0.3596206,-1.9417118,-0.8000972,0.2,0.009067467
4,-2.5739024,-0.8338202,-0.8613619,1.0,0.002091908
5,-0.5364081,-0.8442296,0.5731916,0.2,0.008063160
6,0.9836179,-0.9878642,0.3748360,1.0,0.001528506


In [423]:
cor.test(d[,5], d[,4])


	Pearson's product-moment correlation

data:  d[, 5] and d[, 4]
t = -21.406, df = 498, p-value < 2.2e-16
alternative hypothesis: true correlation is not equal to 0
95 percent confidence interval:
 -0.7353029 -0.6436279
sample estimates:
       cor 
-0.6922479 


In [424]:
cor.test(d[,5], d[,3])


	Pearson's product-moment correlation

data:  d[, 5] and d[, 3]
t = 1.3118, df = 498, p-value = 0.1902
alternative hypothesis: true correlation is not equal to 0
95 percent confidence interval:
 -0.02916002  0.14562187
sample estimates:
       cor 
0.05868061 


In [425]:
cor.test(d[,5], d[,2])


	Pearson's product-moment correlation

data:  d[, 5] and d[, 2]
t = -0.37971, df = 498, p-value = 0.7043
alternative hypothesis: true correlation is not equal to 0
95 percent confidence interval:
 -0.10454723  0.07078358
sample estimates:
        cor 
-0.01701261 


In [426]:
cor.test(d[,5], d[,1])


	Pearson's product-moment correlation

data:  d[, 5] and d[, 1]
t = 0.99371, df = 498, p-value = 0.3208
alternative hypothesis: true correlation is not equal to 0
95 percent confidence interval:
 -0.04337455  0.13166222
sample estimates:
       cor 
0.04448524 


In [428]:
png("scatterplot.png", width = 250, height = 250, units='mm', res = 300)
pairs(d,
      col = "pink4",                                         # Change color
      pch = 16,                                            # Change shape of points
      labels = c("x", "y", "carga", "masa", "velocidad"))
dev.off()

pdf 
  2

In [382]:
p[,c(1,2,3,4,6)]->W